In [1]:
from haystack import Pipeline
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack_integrations.document_stores.mongodb_atlas import MongoDBAtlasDocumentStore
from haystack_integrations.components.retrievers.mongodb_atlas import MongoDBAtlasEmbeddingRetriever
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator
from haystack.utils import Secret
import os
from getpass import getpass

/Users/mhdfarhanali/Documents/Farhan SmartShopper/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Environment Setup
os.environ["MONGO_CONNECTION_STRING"] = getpass("Masukkan MongoDB Connection String Anda: ")
os.environ["OPENAI_API_KEY"] = getpass("Masukkan OpenAI API Key Anda: ")

In [3]:
# Connect to MongoDB Atlas

document_store = MongoDBAtlasDocumentStore(
    database_name="depato_store",
    collection_name="common_information",
    vector_search_index="vector_index_common",
    full_text_search_index=None
)

print("Connected to MongoDB Atlas successfully!")

Connected to MongoDB Atlas successfully!


In [4]:
# Build Pipeline

TEMPLATE = """
You are a helpful customer support assistant. Answer the user's question based only on the retrieved context below.

If the context does not provide enough information, politely say you don’t have that information.

Context:
{% for doc in documents %}
{{ doc.content }}
{% endfor %}

Question: {{ query }}
Answer:
"""

pipeline = Pipeline()

pipeline.add_component(
    "embedder",
    SentenceTransformersTextEmbedder(model="sentence-transformers/all-mpnet-base-v2")
)

pipeline.add_component(
    "retriever",
    MongoDBAtlasEmbeddingRetriever(document_store=document_store, top_k=5)
)

pipeline.add_component(
    "prompt_builder",
    PromptBuilder(template=TEMPLATE, required_variables=["query", "documents"])
)

pipeline.add_component(
    "generator",
    OpenAIGenerator(model="gpt-4.1", api_key=Secret.from_token(os.environ["OPENAI_API_KEY"]))
)

# Connect components
pipeline.connect("embedder", "retriever")
pipeline.connect("retriever", "prompt_builder.documents")
pipeline.connect("prompt_builder", "generator")

print("Common Help Center pipeline built successfully!")

Common Help Center pipeline built successfully!


In [5]:
# Test Query

query = "What payment methods do you accept?"

response = pipeline.run(
    {
        "embedder": {"text": query},
        "prompt_builder": {"query": query}
    }
)

print("\nUser Question:", query)
print("Assistant Answer:\n", response["generator"]["replies"][0])

Batches: 100%|██████████| 1/1 [00:00<00:00,  8.76it/s]



User Question: What payment methods do you accept?
Assistant Answer:
 We accept various payment methods including credit/debit cards, bank transfers, e-wallets, and Cash on Delivery (COD) for eligible areas.
